In [4]:
import pandas as pd
import numpy as np
import math
from darts.models import NBEATSModel, RNNModel, TCNModel, TransformerModel
from darts.timeseries import TimeSeries as ts
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mae, rmse,mape,mse
import pickle

The StatsForecast module could not be imported. To enable support for the StatsForecastAutoARIMA, StatsForecastAutoETS and Croston models, please consider installing it.


In [5]:
df = pd.read_csv('San_Juan_Dengue_Data_Cleaned.csv')
df

,week_start_date,total_cases
0,1990-04-30,4
1,1990-05-07,5
2,1990-05-14,4
3,1990-05-21,3
4,1990-05-28,6
...,...,...
983,2009-03-26,24
984,2009-04-02,9
985,2009-04-09,21
986,2009-04-16,10


In [6]:
# Ensure the 'week_start_date' column is in datetime format
df['week_start_date'] = pd.to_datetime(df['week_start_date'])

# Set the 'week_start_date' as the index and enforce a 7-day frequency
df.set_index('week_start_date', inplace=True)

# Use asfreq with forward fill to handle any potential missing data points
df = df.asfreq('7D', method='ffill')

# Reset the index to pass it to your ts object
df.reset_index(inplace=True)

# Now pass the dataframe to your ts object
ts_dengue = ts.from_dataframe(df, time_col='week_start_date', value_cols='total_cases')


In [7]:
ts_dengue

<TimeSeries (DataArray) (week_start_date: 991, component: 1, sample: 1)>
array([[[  4.]],

       [[  5.]],

       [[  4.]],

       [[  3.]],

       [[  6.]],

       [[  2.]],

       [[  4.]],

       [[  5.]],

       [[ 10.]],

       [[  6.]],

...

       [[ 41.]],

       [[ 35.]],

       [[ 28.]],

       [[ 28.]],

       [[ 22.]],

       [[ 26.]],

       [[ 24.]],

       [[  9.]],

       [[ 21.]],

       [[ 10.]]])
Coordinates:
  * week_start_date  (week_start_date) datetime64[ns] 1990-04-30 ... 2009-04-20
  * component        (component) object 'total_cases'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [8]:
with open('Finalscaler.pkl', 'rb') as file:
    loaded_scaler = pickle.load(file)

In [9]:
ts_dengue_scaled = loaded_scaler.fit_transform(ts_dengue)
ts_dengue_train = ts_dengue_scaled[:971]
ts_dengue_val = ts_dengue_scaled[971:981]

In [10]:
with open('TrainedModel.pkl', 'rb') as file:
   loaded = pickle.load(file)
loaded

NBEATSModel(output_chunk_shift=0, generic_architecture=True, num_stacks=30, num_blocks=1, num_layers=4, layer_widths=256, expansion_coefficient_dim=5, trend_polynomial_degree=2, dropout=0.0, activation=LeakyReLU, input_chunk_length=10, output_chunk_length=5, n_epochs=100, model_name=Nbeats100Epochs, save_checkpoints=True, force_reset=True)

HyperParameter-Tuning

In [11]:
parameters = {
    'num_stacks': [30, 50],
    'num_blocks': [1, 3],
    'num_layers': [4, 6],
    'layer_widths': [512, 1024],
    'expansion_coefficient_dim': [5, 10, 15],
    'trend_polynomial_degree': [2, 4],
    'dropout': [0.1, 0.2],
    'input_chunk_length': [15, 30, 45],
    'output_chunk_length': [5, 10, 15],
    'activation': ['ReLU', 'LeakyReLU'],
    'n_epochs': [50, 100],
    'optimizer_kwargs': [{'lr': 1e-3},{'lr': 1e-2}],
    'batch_size': [32, 64]
}

In [12]:
best_model = loaded.gridsearch(
    parameters=parameters,
    series=ts_dengue_train,
    val_series=ts_dengue_val,
    metric=mae,
    verbose=True
)
print(f"Best model: {best_model}")

  0%|          | 0/27648 [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 24.0 M | train
-------------------------------------------------------------
24.0 M    Trainable params
2.7 K     Non-trainable params
24.0 M    Total params
96.169    Total estimated model params size (MB)
486       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 24.0 M | train
-------------------------------------------------------------
24.0 M    Trainable params
2.7 K     Non-trainable params
24.0 M    Total params
96.173    Total estimated model params size (MB)
486       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

: 